## Importing all the required modules and dependencies

In [1]:
import sys
sys.path.append('..')

import torch
from utils.tacotron import Tacotron
from pathlib import Path
from typing import Union, List
import numpy as np
import librosa
from synthesizer import Synthesizer
# from speech_encoder import SpeechEncoder
from speech_encoder_v2_updated import SpeechEncoderV2
# from data_preprocessing import *
from temp.audio import preprocess_wav
import torchaudio
from embed import Embed

### Preprocessing the audio

In [2]:
wav, sample_rate = librosa.load(r"..\2764-36616-0000.flac", sr = 16000)
wav = preprocess_wav(wav)

## Initializing the speaker encoder model and loading the checkpoints

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loss_device = torch.device("cpu")

# encoder = SpeechEncoder(device, loss_device)
encoder = SpeechEncoderV2(device, device)

# checkpoints = torch.load("..\models\speech_encoder_lstm\encoder.pt")
checkpoints = torch.load("..\models\speech_encoder_transformer_updated\encoder_014000_loss_0.4987.pt")

encoder.load_state_dict(checkpoints['model_state'])
embedder = Embed(encoder)

c:\Users\athar\anaconda3\envs\motive_3.10\lib\site-packages\torch\nn\modules\transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


## Generating the speaker embeddings

In [4]:
embedding, partial_embeds, _ = embedder.embed_utterance(wav, return_partials=True)
# embedding = np.expand_dims(embedding, 0)
text = "I went to the zoo with my family, we saw birds, lions and various other animals. WE enjoyed the day.".split("\n")
embeddings = [embedding] * len(text)

In [5]:
synthesizer_model_path = Path("..\models/synthesizer/synthesizer.pt")
synthesizer = Synthesizer(synthesizer_model_path)

synthesizer.load()
specs = synthesizer.synthesize_spectrograms(text, embeddings)
spec = np.concatenate(specs, axis=1)

breaks = [spec.shape[1] for spec in specs]

Synthesizer using device: cuda
Trainable Parameters: 30.870M
Loaded synthesizer "synthesizer.pt" trained to step 295000

| Generating 1/1


Done.



## Generating WAV using the Griffiin Lim Algorithm

In [ ]:
wav_og = synthesizer.griffin_lim(spec)

In [ ]:
b_ends = np.cumsum(np.array(breaks) * Synthesizer.params.hop_size)
b_starts = np.concatenate(([0], b_ends[:-1]))
wavs = [wav_og[start:end] for start, end, in zip(b_starts, b_ends)]
breaks = [np.zeros(int(0.15 * Synthesizer.sample_rate))] * len(breaks)
wav = np.concatenate([i for w, b in zip(wavs, breaks) for i in (w, b)])

In [ ]:
# wav_og = preprocess_audio(wav_og, Synthesizer.sample_rate)
wav_processed = wav_og / np.abs(wav_og).max() * 0.97

import IPython.display as ipd
ipd.Audio(wav, rate=16000)

## Generating WAV using the Vocoder (Corentin Jemine)

In [6]:
from vocoder import Vocoder

vocoder = Vocoder()
vocoder.load_model("..\models/vocoder/vocoder.pt")

Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at ..\models/vocoder/vocoder.pt


In [7]:
wav_new = vocoder.infer_waveform(spec)

b_ends = np.cumsum(np.array(breaks) * Synthesizer.params.hop_size)
b_starts = np.concatenate(([0], b_ends[:-1]))
wavs = [wav_new[start:end] for start, end, in zip(b_starts, b_ends)]
breaks = [np.zeros(int(0.15 * Synthesizer.sample_rate))] * len(breaks)
wav_vocoder = np.concatenate([i for w, b in zip(wavs, breaks) for i in (w, b)])

wav_vocoder = wav_new / np.abs(wav_new).max() * 0.97    

import IPython.display as ipd
ipd.Audio(wav_new, rate=16000)

{| ████████████████ 57000/57600 | Batch Size: 6 | Gen Rate: 1.8kHz | }